In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Input, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
import os
import datetime
import time
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tqdm.notebook import tqdm
print('ライブラリのインポート完了')



WTP = ['2-MIB', 'Geosmin']
for wtp in tqdm(WTP, desc='wtp'):

    TS = [24]
    F = [3]
    M = [25]

    for m in tqdm(M, desc='m'):

        for ts in tqdm(TS, desc='ts'):

            for f in tqdm(F, desc='f'):

                df1 = pd.read_csv('./csv/train.csv')
                df2 = pd.read_csv('./csv/val.csv')
                df3 = pd.read_csv('./csv/test.csv')
                
                n_in = df1.shape[1]
                n_hidden = 1024
                n_out = 1

                epochs = 50
                batch_size = 64

                data1 = []
                target1 = []
                data2 = []
                target2 = []
                data3 = []
                target3 = []
                
                time_steps = ts
                future = f

                df_save = df1
                
                df = df1
                df11 = (df1 - df_save.min()) / (df_save.max() - df_save.min())
                df = pd.concat([df[wtp]], axis=1)
                df1 = pd.DataFrame(df)

                for i in range(len(df11) - time_steps - future):
                    data1.append(df11.values[i: i + time_steps])

                for i in range(len(df1) - time_steps - future):
                    target1.append(df1.values[i + time_steps + future])

                x_train = np.array(data1)
                y_train = np.array(target1)

                x_train = x_train.reshape(len(x_train), time_steps, n_in)
                y_train = y_train.reshape(len(y_train), n_out)


                df = df2
                df22 = (df2 - df_save.min()) / (df_save.max() - df_save.min())
                df = pd.concat([df[wtp]], axis=1)
                df2 = pd.DataFrame(df)

                for i in range(len(df22) - time_steps - future):
                    data2.append(df22.values[i: i + time_steps])

                for i in range(len(df2) - time_steps - future):
                    target2.append(df2.values[i + time_steps + future])

                x_val = np.array(data2)
                y_val = np.array(target2)

                x_val = x_val.reshape(len(x_val), time_steps, n_in)
                y_val = y_val.reshape(len(y_val), n_out)

                df = df3
                df33 = (df3 - df_save.min()) / (df_save.max() - df_save.min())
                df = pd.concat([df[wtp]], axis=1)
                df3 = pd.DataFrame(df)


                for i in range(len(df33) - time_steps - future):
                    data3.append(df33.values[i: i + time_steps])

                for i in range(len(df3) - time_steps - future):
                    target3.append(df3.values[i + time_steps + future])
                x_test = np.array(data3)
                y_test = np.array(target3)

                x_test = x_test.reshape(len(x_test), time_steps, n_in)
                y_test = y_test.reshape(len(y_test), n_out)

                ES = EarlyStopping(monitor='val_loss', patience=4, verbose=1)


                def step_decay(epoch):
                    x = 0.001
                    if epoch >= 50: x = 0.0001
                    if epoch >= 100: x = 0.00001
                    if epoch >= 300: x = 0.000001
                    return x


                lr_decay = LearningRateScheduler(step_decay)



                input = Input(shape=(time_steps, n_in))

                x = LSTM(n_hidden)(input)
                # x = Dropout(0.3)(x, training=True)
                output = Dense(n_out, name="Output_Layer")(x)
                model = Model(input, output)



                optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
                model.compile(loss="mse", optimizer=optimizer)

                reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=3, factor=0.1, min_lr=1e-6,
                                              mode='auto', verbose=1)

                new_folder = wtp + "_ts" + str(ts) + "_f" + str(f) + "_data" + str(m) 
                if not os.path.exists(new_folder): 
                    os.mkdir(new_folder) 



                model_checkpoint = ModelCheckpoint(
                    filepath=os.path.join(new_folder, 'model.h5'),
                    monitor='val_loss',
                    save_best_only=True,
                    save_weights_only=True,
                    verbose=1)

                history = model.fit(x_train, y_train,
                                     batch_size=batch_size,
                                     epochs=epochs,
                                     verbose=1,
                                     validation_data=(x_val, y_val),
                                     callbacks=[ES, reduce_lr, model_checkpoint]
                                     )


                open(new_folder + '/' + 'model.json', "w").write(model.to_json())
                model.save_weights(new_folder + '/' + 'last_model.h5')

                json_file = open(new_folder + '/' + 'model.json', 'r')
                loaded_model_json = json_file.read()
                json_file.close()
                model = model_from_json(loaded_model_json)

                weightPath = new_folder + '/' + 'model.h5'
                if not os.path.exists(new_folder):
                    model.load_weights(new_folder + '/' + 'last_model.h5')
                else:
                    model.load_weights(new_folder + '/' + 'model.h5')


                optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
                model.compile(loss="mse", optimizer=optimizer)

                train_predict = model.predict(x_train)
                train_predict = pd.DataFrame(train_predict)
                train_predict.to_csv(new_folder + '/' + 'train_predict_.csv')
                y_train = pd.DataFrame(y_train)
                y_train.to_csv(new_folder + '/' +'y_train.csv')


                plt.plot(y_train, label='train', color='black', linewidth=0.7)
                plt.plot(train_predict, label='pred', color='blue', linewidth=0.5)
                plt.title("train")
                plt.xlim(0, 2000)
                plt.savefig(new_folder + '/' + 'LSTM_train.pdf')
                plt.show()


                plt.scatter(y_train, train_predict)
                plt.title("train")
                plt.xlim(0, 100)
                plt.ylim(0, 100)
                plt.savefig(new_folder + '/' + 'train_scatter.pdf')
                plt.show()

                val_predict = model.predict(x_val)
                val_predict = pd.DataFrame(val_predict)
                val_predict.to_csv(new_folder + '/' + 'val_predict_.csv')
                y_val = pd.DataFrame(y_val)
                y_val.to_csv(new_folder + '/' + 'y_val.csv')

                plt.plot(y_val, label='val', color='black', linewidth=0.7)
                plt.plot(val_predict, label='pred', color='blue', linewidth=0.5)
                plt.title("val")
                plt.xlim(0, 2000)
                plt.savefig(new_folder + '/' + 'LSTM_val.pdf')
                plt.show()


                plt.scatter(y_val, val_predict)
                plt.title("val")
                plt.xlim(0, 100)
                plt.ylim(0, 100)
                plt.savefig(new_folder + '/' + 'val_scatter.pdf')
                plt.show()



                test_predict = model.predict(x_test)
                test_predict = pd.DataFrame(test_predict)
                test_predict.to_csv(new_folder + '/' + 'test_predict.csv')
                y_test = pd.DataFrame(y_test)
                y_test.to_csv(new_folder + '/' + 'y_test.csv')


                plt.plot(y_test, label='test', color='black', linewidth=0.7)
                plt.plot(test_predict, label='pred', color='blue', linewidth=0.5)
                plt.title("test")
                plt.xlim(0, 2000)
                plt.savefig(new_folder + '/' + 'LSTM_test.pdf')
                plt.show()


                plt.scatter(y_test, test_predict)
                plt.title("test")
                plt.xlim(0, 100)
                plt.ylim(0, 100)
                plt.savefig(new_folder + '/' + 'test_scatter.pdf')
                plt.show()

                loss = history.history['loss']
                val_loss = history.history['val_loss']
                epochs = len(loss)
                plt.plot(range(epochs), loss, marker='.', label='loss')
                plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
                plt.legend(loc='best')
                plt.grid()
                plt.xlabel('epoch')
                plt.ylabel('loss')
                plt.savefig(new_folder + '/' + 'LSTM_epochs.pdf')
                plt.show()




